In [7]:
import os
import xml.etree.ElementTree as ET

def convert(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

def convert_annotations(xml_folder, txt_folder):
    classes = ['plate']  # Sesuaikan dengan kelas dalam file XML Anda
    if not os.path.exists(txt_folder):
        os.makedirs(txt_folder)
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(xml_folder, xml_file))
            root = tree.getroot()
            size = root.find('size')
            w = int(size.find('width').text)
            h = int(size.find('height').text)

            txt_file = xml_file.replace('.xml', '.txt')
            with open(os.path.join(txt_folder, txt_file), 'w') as file:
                for obj in root.findall('object'):
                    cls = obj.find('name').text
                    if cls not in classes:
                        continue
                    cls_id = classes.index(cls)
                    xmlbox = obj.find('bndbox')
                    b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                         float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
                    bb = convert((w, h), b)
                    file.write(str(cls_id) + " " + " ".join(map(str, bb)) + '\n')

# Jalankan fungsi konversi untuk train dan test
base_folder = 'D:\\studeh\\Bangkit\\kepston\\dataset\\annotations'
convert_annotations(os.path.join(base_folder, 'train'), 'D:\\studeh\\Bangkit\\kepston\\dataset\\label\\train')
convert_annotations(os.path.join(base_folder, 'test'), 'D:\\studeh\\Bangkit\\kepston\\dataset\\label\\test')

In [1]:
%cd "D:\studeh\Bangkit\kepston\yolov5"
!python train.py --img 640 --batch 16 --epochs 30 --data data.yaml --weights yolov5s.pt --cache


D:\studeh\Bangkit\kepston\yolov5
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5


train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
YOLOv5  v7.0-386-g81ac034a Python-3.12.4 torch-2.5.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degr

In [ ]:
# Object Tracking with Deep Sort
import cv2
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort

# Memuat model YOLOv5s yang sudah dilatih
model = torch.hub.load('ultralytics/yolov5', 'custom', path='D:\\studeh\\Bangkit\\kepston\\yolov5\\runs\\train\\exp6\\weights\\best.pt', force_reload=True)

# Inisialisasi Deep SORT
deepsort = DeepSort(max_age=30, n_init=3, nn_budget=100)

# Buka video input
cap = cv2.VideoCapture("videotest.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Deteksi objek dengan YOLOv5
    results = model(frame)
    
    # Mendapatkan bounding boxes, confidence score, dan class ids
    boxes = results.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2, confidence, class_id]
    confidences = boxes[:, 4]  # Ambil confidence dari kolom ke-4
    class_ids = boxes[:, 5]  # Ambil class ID dari kolom ke-5

    # Tampilkan dimensi hasil deteksi (debugging)
    print("Detected boxes:", boxes.shape)
    
    # Filter hanya untuk plat nomor (ID class untuk plat nomor adalah 0)
    filtered_boxes = []
    filtered_confidences = []
    for i, class_id in enumerate(class_ids):
        if class_id == 0:  # Ganti dengan class ID plat nomor (0 untuk plat nomor)
            filtered_boxes.append(boxes[i][:4])  # Ambil hanya [x1, y1, x2, y2] (4 elemen pertama)
            filtered_confidences.append(confidences[i])  # Ambil confidence yang sesuai

    # Menampilkan bounding box hasil deteksi dari YOLOv5 (debugging)
    for box in filtered_boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Pastikan bahwa filtered_boxes tidak kosong
    if filtered_boxes:
        # Tracking dengan Deep SORT
        # Format yang diterima Deep SORT: [x1, y1, x2, y2, score]
        detections = []
        for i, box in enumerate(filtered_boxes):
            x1, y1, x2, y2 = box
            # Pastikan deteksi adalah list dengan format [x1, y1, x2, y2, confidence]
            detection = [float(x1), float(y1), float(x2), float(y2), float(filtered_confidences[i])]
            detections.append(detection)

        # Debugging: Tampilkan deteksi
        print(f"Detections: {detections}")

        # Update tracking
        tracker = deepsort.update_tracks(detections, frame=frame)
        
        # Visualisasi tracking
        for track in tracker:
            x1, y1, x2, y2 = track.to_tlbr()  # Ambil koordinat bounding box
            track_id = track.track_id  # ID untuk setiap track (objek yang terdeteksi dan dilacak)

            # Gambar bounding box dan ID
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX,

SyntaxError: incomplete input (988165095.py, line 67)